## Live Coding Module 6

#### H. Diana McSpadden

This week we will be wrangling data from the latest season of the National Basketball Association into a relational database with five tables:

1. Information overall about the game: OT, date, location, etc
1. Info about how the team overall did in the game
1. Info about how the player did personally in the game
1. Info about the team's total season stats so far
1. Info about the player's total season so far


The pandas code to generate these tables is here:
* http://jkropko.github.io/nba_tables2.html


Start by copy and pasting this code into your own notebook.

If you have trouble with the website, get (older) data hereDownload If you have trouble with the website, get (older) data here

Here is more detail and background for this week's live coding problem:

In [3]:
import numpy as np
import pandas as pd
import psycopg2
import dotenv
import os
from sqlalchemy import create_engine

Note: the URL is changing every few hours. First go to https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/ then right click on "Raw Data Download" and copy and paste the new URL below:

In [2]:
#url = 'https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/_w_3d70b077/session/2d2c295ce7b43033b1daffdbbefdfc29/download/downloadData?w=3d70b077'

In [10]:
nba = pd.read_csv('ASA_All_NBA_Raw_Data.csv')


C:\Users\dianam\AppData\Local\Temp\ipykernel_31672\3279600496.py:1: DtypeWarning: Columns (68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  nba = pd.read_csv('ASA_All_NBA_Raw_Data.csv')


In [6]:
pd.set_option('display.max_columns', None)

In [8]:
nba.head(2)

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Inactives,Opponent_Abbrev,Opponent_Score,Opponent_pace,Opponent_efg_pct,Opponent_tov_pct,Opponent_orb_pct,Opponent_ft_rate,Opponent_off_rtg,player,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,bpm,season,minutes,double_double,triple_double,DKP,FDP,SDP,DKP_per_minute,FDP_per_minute,SDP_per_minute,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,0.157,123.8,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,Corey Kispert,kispeco01,1,32:30,6,9,0.667,4,6,0.667,0,0,0.0,0,1,1,1,2,0,2,2,16,13,0,0,0.889,0.889,0.667,0.000,0.0,3.3,1.8,4.0,3.1,0.0,18.2,15.6,133,113,3.1,2022,32.500000,0,0,23.75,22.7,24.75,0.730769,0.698462,0.761538,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,0.157,123.8,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,Kyle Kuzma,kuzmaky01,1,30:16,2,7,0.286,0,3,0.000,1,1,1.0,0,7,7,5,1,0,7,3,5,10,0,0,0.336,0.286,0.429,0.143,0.0,24.7,13.9,18.5,1.7,0.0,48.5,22.0,55,107,-15.5,2022,30.266667,0,0,19.75,16.9,17.25,0.652533,0.55837,0.569934,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590


In [12]:
pd.options.display.max_rows = 100
nba.head(2).T

,0,1
game_id,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17
OT,0,0
H_A,A,A
Team_Abbrev,WAS,WAS
Team_Score,117,117
Team_pace,94.5,94.5
Team_efg_pct,0.627,0.627
Team_tov_pct,13.5,13.5
Team_orb_pct,22.9,22.9


In [ ]:
pd.options.display.max_rows = 10
nba = nba.drop('Inactives', axis=1)

## Normal Form Rules

### First Normal Form
1. primary key (one, or multiple columns (superkey))
2. atomic
3. no repeating groups

**Inactives** are **NOT** atomic

### Second Normal Form
1. first normal form

### Third Normal Form
1. second normal form

Five tables

1. Information overall about the game: OT, date, location, etc

1. Info about how the team overall did in the game

1. Info about how the player did personally in the game

1. Info about the team's total season stats so far

1. Info about the player's total season so far

### First we filter the data to just the current NBA season

In [ ]:
nba2022 = nba.query("season==2022")

### Information overall about the game: OT, date, location, etc

In [ ]:
game_info = nba2022[['game_id', 'game_date', 'OT']].drop_duplicates()
game_info

### Info about how the team overall did in the game

In [ ]:
nba2022['win'] = nba2022['Team_Score'] > nba2022['Opponent_Score']
team_game = nba2022[['Team_Abbrev', 'H_A', 'win', 'game_id', 'fg', 'fga', 'fg3', 'fg3a', 
             'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']]
team_game = team_game.groupby(['game_id','Team_Abbrev']).agg({'H_A': pd.Series.mode,
                                                              'win': 'mean',
                                                              'fg': sum,
                                                              'fga': sum, 
                                                              'fg3': sum, 
                                                              'fg3a': sum, 
                                                              'ft': sum, 
                                                              'fta': sum, 
                                                              'orb': sum, 
                                                              'drb': sum, 
                                                              'ast': sum, 
                                                              'stl': sum, 
                                                              'blk': sum, 
                                                              'tov': sum, 
                                                              'pf': sum, 
                                                              'pts': sum})
team_game.reset_index()

### Info about how the player did personally in the game

In [ ]:
player_game = nba2022[['game_id', 'player_id', 'starter', 'minutes', 'fg', 'fga', 'fg3', 'fg3a', 
                   'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'usg_pct', 
                   'is_inactive', 'PG%', 'SG%', 'SF%', 'PF%', 'C%']]
player_game

### Info about the team's total season stats so far

In [ ]:
teams = nba2022[['Team_Abbrev', 'game_id', 'fg', 'fga', 'fg3', 'fg3a', 
             'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']]
teams = teams.groupby('Team_Abbrev').agg({'fg': sum,
                                          'fga': sum, 
                                          'fg3': sum, 
                                          'fg3a': sum, 
                                          'ft': sum, 
                                          'fta': sum, 
                                          'orb': sum, 
                                          'drb': sum, 
                                          'ast': sum, 
                                          'stl': sum, 
                                          'blk': sum, 
                                          'tov': sum, 
                                          'pf': sum, 
                                          'pts': sum})
WL = nba2022.groupby(['Team_Abbrev', 'game_id']).agg({'win': 'mean'})
WL = WL.groupby('Team_Abbrev').agg({'win':[sum, 'count']})
WL.columns = ['wins', 'totalgames']
WL['losses'] = WL['totalgames'] - WL['wins']
teams = pd.merge(teams, WL, on=['Team_Abbrev'], validate='one_to_one')
teams = teams.reset_index()
teams

### Info about the player's total season so far

In [ ]:
players = nba2022[['player', 'player_id', 'starter', 'minutes', 'fg', 'fga', 'fg3', 'fg3a', 
                   'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'usg_pct', 
                   'did_not_play', 'is_inactive', 'ts_pct', 'PG%', 'SG%', 'SF%', 'PF%', 'C%']]
players = players.groupby(['player','player_id']).agg({'starter': sum,
                                                   'minutes': 'mean',
                                                   'fg': sum,
                                                   'fga': sum, 
                                                   'fg3': sum, 
                                                   'fg3a': sum, 
                                                   'ft': sum, 
                                                   'fta': sum, 
                                                   'orb': sum, 
                                                   'drb': sum, 
                                                   'ast': sum, 
                                                   'stl': sum, 
                                                   'blk': sum, 
                                                   'tov': sum, 
                                                   'pf': sum, 
                                                   'pts': sum,
                                                  'usg_pct': 'mean',
                                                   'ts_pct': 'mean',
                                                  'did_not_play': sum,
                                                  'is_inactive': sum,
                                                  'PG%': 'mean', 
                                                   'SG%': 'mean', 
                                                   'SF%': 'mean', 
                                                   'PF%': 'mean', 
                                                   'C%': 'mean'}).add_prefix('season_')
players.reset_index()